## Importing libraries

In [34]:
# importing libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')


Libraries imported.


In [35]:
# Reading wiki page to pandas dataframe
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link,header=0,attrs={"class":"wikitable sortable"})[0]

In [36]:
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### filtering data and removind the buroughs that has not assigned name

In [37]:
# filtering data and removind the buroughs that has not assigned name
df_filtered = tables[tables['Borough'] !='Not assigned'].copy()

In [38]:
df_filtered.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### grouping all boroughs with same zip code

In [6]:
# grouping all boroughs with same zip code
df_group=df_filtered.groupby(['Postcode','Borough']).agg( ','.join).reset_index()

In [39]:
df_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### removing neighborhood that has not assigned name and giveing it the same name as burough

In [40]:
# removing neighborhood that has not assigned name and giveing it the same name as burough
df_filtered['Neighbourhood']=df_filtered['Neighbourhood'].apply(lambda x: "Queen's Park" if x=='Not assigned' else x)

In [41]:
df_filtered

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [42]:
df_group.shape

(103, 3)

###### reading geospatial coordinates csv that has latitudes and longitudes to pandas

In [43]:
# reading geospatial coordinates csv that has latitudes and longitudes to pandas
df_geo=pd.read_csv('Geospatial_Coordinates.csv')

In [44]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### renaming postal code column to postcode to manin same column names on both data sets

In [46]:
#renaming postal code column to postcode to manin same column names on both data sets
df_geo=df_geo.rename(columns={"Postal Code": "Postcode"})
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### merge both data sents on postcode column

In [47]:
# merge both data sents on postcode column
df_complete = pd.merge(df_filtered, df_geo, on='Postcode')
df_complete

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [59]:
df_complete=df_complete.groupby(['Postcode','Borough']).agg( ','.join).reset_index()

In [60]:
df_complete = pd.merge(df_complete, df_geo, on='Postcode')

##### Now I have started working on bring out "North York" data

In [61]:
# Now I have started working on bring out "North York" data
df_northyork=df_complete[df_complete['Borough']=="North York"].reset_index()

In [62]:
df_northyork

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,17,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,18,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,19,M2K,North York,Bayview Village,43.786947,-79.385975
3,20,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,21,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493
5,22,M2N,North York,Willowdale South,43.770120,-79.408493
6,23,M2P,North York,York Mills West,43.752758,-79.400049
7,24,M2R,North York,Willowdale West,43.782736,-79.442259
8,25,M3A,North York,Parkwoods,43.753259,-79.329656
9,26,M3B,North York,Don Mills North,43.745906,-79.352188


##### first find out geographical locations for Toronto

In [63]:
# Since North York is in toronto, first find out geographical locations for Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


##### create map of Toronto using latitude and longitude values

In [64]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Borough'], df_complete['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

##### Finding out latitudes and logitudes for North York

In [65]:
# Finding out latitudes and logitudes for North York
address = 'North York, Toronto'
geolocator = Nominatim(user_agent="ca_explorer")
location =geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


##### creating a map for Nort York

In [66]:
#creating a map for Nort York
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_northyork['Latitude'], df_northyork['Longitude'], df_northyork['Borough'], df_northyork['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

#### Bringing out Forusquare credentials

In [67]:
#Bringing out Forusquare credentials
CLIENT_ID = 'XMFFMVMS0USJX11FNRVQFMMBMWDWMQDHJ51EOQG5RUM5FVMR' # your Foursquare ID
CLIENT_SECRET = 'NBMPMO0C3BX2E3CEYEGQ1RSPIGWFDU0LMTGGBNX5JZBSTGNU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XMFFMVMS0USJX11FNRVQFMMBMWDWMQDHJ51EOQG5RUM5FVMR
CLIENT_SECRET:NBMPMO0C3BX2E3CEYEGQ1RSPIGWFDU0LMTGGBNX5JZBSTGNU


In [68]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XMFFMVMS0USJX11FNRVQFMMBMWDWMQDHJ51EOQG5RUM5FVMR&client_secret=NBMPMO0C3BX2E3CEYEGQ1RSPIGWFDU0LMTGGBNX5JZBSTGNU&v=20180605&ll=43.7708175,-79.4132998&radius=500&limit=100'

In [69]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d16c0422081a2002b29f270'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Willowdale',
  'headerFullLocation': 'Willowdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 52,
  'suggestedBounds': {'ne': {'lat': 43.7753175045, 'lng': -79.40707971454917},
   'sw': {'lat': 43.7663174955, 'lng': -79.41951988545084}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '563d44fccd1044ad67a744fb',
       'name': "The Captain's Boil",
       'location': {'address': '5313 Yonge St',
        'lat': 43.773255217045026,
        'lng': -79.41380541792645,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7732552170

In [70]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
# get nearby venues
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Captain's Boil,Seafood Restaurant,43.773255,-79.413805
1,Aroma Espresso Bar,Café,43.769449,-79.413081
2,Loblaws,Grocery Store,43.768648,-79.412597
3,Starbucks,Coffee Shop,43.768192,-79.413021
4,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613


In [72]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

52 venues were returned by Foursquare.


In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
              
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get venues for each neighborhood in North York

In [74]:
northyork_venues = getNearbyVenues(names=df_northyork['Neighbourhood'],
                                   latitudes=df_northyork['Latitude'],
                                   longitudes=df_northyork['Longitude']
                                  )

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,North Park,Upwood Park
Humber Summit
Emery,Humberlea


In [75]:
print(northyork_venues.shape)
northyork_venues.head()

(251, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [76]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto,Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,3,3,3,3,3,3
"Downsview,North Park,Upwood Park",5,5,5,5,5,5


In [77]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 111 uniques categories.


In [78]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [79]:
northyork_onehot.shape

(251, 112)

##### Onehot encoding

In [81]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,...,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.0000,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.040000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto,Downsview East",0.0000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.0000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview,North Park,Upwood Park",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery,Humberlea",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [82]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1                Diner  0.06
2        Shopping Mall  0.06
3           Restaurant  0.06
4  Fried Chicken Joint  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3              Pharmacy  0.04
4        Sandwich Place  0.04


----CFB Toronto,Downsview East----
               venue  freq
0            Airport  0.33
1               Park  0.33
2  Other Repair Shop  0.33
3                Gym  0.00
4          Pet Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Get top 10 venues per neighborhood

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Indian Restaurant,Liquor Store,Cupcake Shop,Café,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


## Run k-means to cluster the neighborhoods into 5 clusters

In [85]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.shape

(23,)

## Include kmeans.labels_ into the original North York dataframe

In [86]:
northyork_merged = df_northyork
northyork_merged=northyork_merged.rename(columns={'Neighbourhood':'Neighborhood'})
northyork_merged=northyork_merged.drop(16)
# add clustering labels
northyork_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.merge(neighborhoods_venues_sorted, on='Neighborhood')

northyork_merged

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,17,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,18,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,19,M2K,North York,Bayview Village,43.786947,-79.385975,3,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
3,20,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,22,M2N,North York,Willowdale South,43.770120,-79.408493,3,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
5,23,M2P,North York,York Mills West,43.752758,-79.400049,3,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
6,24,M2R,North York,Willowdale West,43.782736,-79.442259,3,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
7,25,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,26,M3B,North York,Don Mills North,43.745906,-79.352188,4,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
9,27,M3C,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,3,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Smoke Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store


## Visualize the clusters in the map

In [87]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine each of the five clusters

In [88]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M2L,-79.374714,0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
16,M5M,-79.419750,0,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Indian Restaurant,Liquor Store,Cupcake Shop,Café,Pharmacy,Pizza Place
21,M9M,-79.532242,0,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


In [89]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M6B,-79.445073,1,Pizza Place,Pub,Japanese Restaurant,Park,Women's Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [90]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M6A,-79.464763,2,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Fraternity House,Accessories Store,Vietnamese Restaurant


In [48]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,-79.363452,3,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,M2J,-79.346556,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,M2K,-79.385975,3,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,M2N,-79.408493,3,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
5,M2P,-79.400049,3,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
6,M2R,-79.442259,3,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
7,M3A,-79.329656,3,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
9,M3C,-79.340923,3,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Smoke Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
10,M3H,-79.442259,3,Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
11,M3J,-79.487262,3,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Falafel Restaurant,Discount Store,Cosmetics Shop,Cupcake Shop


In [91]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M3B,-79.352188,4,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


 # Steps for this project:
 
 - Imported pandas
 - Read html wiki link to tables data frame. 
     - To do this, I have used pd.read_html. 
     - Since wiki page has lot of data that is not need for this capstone,
         - on the browser, right click on wiki page, click on view sorce code to find class name (wikitable sortable)and passed it to attributes.
 - Filtered and removed "not assigned" from Borough 
 - used groupby and aggregate function to combine neighbourhood with same postal code.
 - Borough Queen's park is assigned to Neighbourhood as well where Neighbourhood name is 'Not assigned' using 'apply' method, 'lambda' function and 'if else'
 - Printed number of rows and columns using shape 
 
 - Read latitudes and logitudes csv file to pandas using pd.read_csv
 - Renamed 'Postal Code' column to 'Postcode column' to make the id column name same
 - Used merge method on postcode to merge both data frames
 
 - Found latitides and longitudes for Canada and Toronto
 - used foursquare api credientials(client is and client secret) and printed the same
 - explored nearby venues and neighborhood
 - Did onehot encodint to convert everything to 0's and 1's
 - created 5 clusters and examined each one of them.